## Creating a Dataframe with pyspark

In [123]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
 

In [124]:
# Spark session created . Basically it is important to manage the DB session, as there are limited threads to these sessions
spark = SparkSession.builder.appName("nlp").getOrCreate()

In [170]:
# Read in data and store in dataframe
# Pyspark can infer schema types by setting inferSchema and header to True
#df = spark.read.csv("./InputData/winemag-data-130k-v2.csv", inferSchema=True, header=True)
df = spark.read.csv("clean_wine_data.csv", inferSchema=True, header=True)

In [171]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Unnamed: 0: string (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: string (nullable = true)
 |-- price: string (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- taster_name: string (nullable = true)
 |-- taster_twitter_handle: string (nullable = true)
 |-- title: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [173]:
# Show dataframe
df.show()

+---+----------+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|_c0|Unnamed: 0|  country|         description|         designation|points|price|         province|           region_1|         region_2|       taster_name|taster_twitter_handle|               title|           variety|             winery|
+---+----------+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|  0|         0|    Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|               Etna|             null|     Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|       White Blend|            Nicosia|
|  1|         1| Portugal|This is ripe and .

In [174]:
# Order a dataframe by ascending values
df.orderBy(df["description"].asc()).select("description").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|description                                                                                                                                                                                                                                                                                                                                                                                                      |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [175]:
df.count()

129975

In [176]:
# Get the count of non-null columns
new_df = df.where(df.description.isNotNull())
new_df

DataFrame[_c0: string, Unnamed: 0: string, country: string, description: string, designation: string, points: string, price: string, province: string, region_1: string, region_2: string, taster_name: string, taster_twitter_handle: string, title: string, variety: string, winery: string]

In [177]:
# Import functions
from pyspark.sql.functions import avg
df.select(avg("points")).show()

+----------------+
|     avg(points)|
+----------------+
|88.4471254462637|
+----------------+



In [178]:
# Using SQL
df.filter("price<20").show()

+---+----------+---------+--------------------+--------------------+------+-----+-----------------+--------------------+-----------------+------------------+---------------------+--------------------+-------------------+--------------------+
|_c0|Unnamed: 0|  country|         description|         designation|points|price|         province|            region_1|         region_2|       taster_name|taster_twitter_handle|               title|            variety|              winery|
+---+----------+---------+--------------------+--------------------+------+-----+-----------------+--------------------+-----------------+------------------+---------------------+--------------------+-------------------+--------------------+
|  1|         1| Portugal|This is ripe and ...|            Avidagos|    87| 15.0|            Douro|                null|             null|        Roger Voss|           @vossroger|Quinta dos Avidag...|     Portuguese Red| Quinta dos Avidagos|
|  2|         2|       US|Tart a

In [180]:
# Filter by price on certain columns
df.filter("price<20").select(['description','variety']).show()

+--------------------+-------------------+
|         description|            variety|
+--------------------+-------------------+
|This is ripe and ...|     Portuguese Red|
|Tart and snappy t...|         Pinot Gris|
|Pineapple rind le...|           Riesling|
|Blackberry and ra...| Tempranillo-Merlot|
|Here's a bright i...|           Frappato|
|Savory dried thym...|     Gewürztraminer|
|Soft supple plum ...| Cabernet Sauvignon|
|Building on 150 y...|         Chardonnay|
|Raw black-cherry ...|             Malbec|
|Delicate aromas r...|        White Blend|
|Pretty aromas of ...|        White Blend|
|Aromas recall rip...|       Nero d'Avola|
|Aromas suggest ma...|          Red Blend|
|Clarksburg is bec...|       Chenin Blanc|
|White flower lych...|Viognier-Chardonnay|
|Inky in color thi...|          Primitivo|
|Part of the natur...|       Nero d'Avola|
|Catarratto is one...|         Catarratto|
|This is a festive...|              Gamay|
|The clean brisk m...|    Sauvignon Blanc|
+----------

## Tokenization

In [181]:
# Show data frame with description and variety columns

#df.select(['description','variety']).show(truncate=False)
df.select(['description','variety']).show()
 

+--------------------+------------------+
|         description|           variety|
+--------------------+------------------+
|Aromas include tr...|       White Blend|
|This is ripe and ...|    Portuguese Red|
|Tart and snappy t...|        Pinot Gris|
|Pineapple rind le...|          Riesling|
|Much like the reg...|        Pinot Noir|
|Blackberry and ra...|Tempranillo-Merlot|
|Here's a bright i...|          Frappato|
|This dry and rest...|    Gewürztraminer|
|Savory dried thym...|    Gewürztraminer|
|This has great de...|        Pinot Gris|
|Soft supple plum ...|Cabernet Sauvignon|
|This is a dry win...|    Gewürztraminer|
|Slightly reduced ...|Cabernet Sauvignon|
|This is dominated...| Nerello Mascalese|
|Building on 150 y...|        Chardonnay|
|Zesty orange peel...|          Riesling|
|Baked plum molass...|            Malbec|
|Raw black-cherry ...|            Malbec|
|Desiccated blackb...| Tempranillo Blend|
|Red fruit aromas ...|          Meritage|
+--------------------+------------

In [182]:
tokenizer = Tokenizer(inputCol="description", outputCol="words")
tokenizer

Tokenizer_43b2b3248c1cef9fe12e

In [183]:
# Transform and show dataframe
tokenized = tokenizer.transform(df )
tokenized.select('words').show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|words                                                                                                                                                                                                                                                                                                                                                                        |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [165]:
a = ["this", "is,", "a", "list."]

In [169]:
[el.replace(',', '').replace('.', '') for el in a]

['this', 'is', 'a', 'list']

In [194]:
#Need to do the stop words before 
#TF-idf (Term Frequency "TF" and Inverse Document Frequency "IDF")
# Remove stop words
 
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

In [185]:
# Transform new dataframe
newFrame = remover.transform(tokenized)
newFrame.show()

+---+----------+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+--------------------+--------------------+
|_c0|Unnamed: 0|  country|         description|         designation|points|price|         province|           region_1|         region_2|       taster_name|taster_twitter_handle|               title|           variety|             winery|               words|            filtered|
+---+----------+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+--------------------+--------------------+
|  0|         0|    Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|               Etna|             null|     Kerin O’Keefe| 

In [186]:
# Show simplified review
newFrame.select("filtered").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|filtered                                                                                                                                                                                                                                                                                |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[aromas, include, tropical, fruit, broom, brimstone, dried, herb, palate, overly, expressive, offering, unripened, apple, citrus, dried, sage, alongsi

In [187]:
# Run the hashing term frequency
hashing = HashingTF(inputCol="filtered", outputCol="hashedValues")
hashing

HashingTF_40e3b459e8076eedfd4e

In [188]:
# Transform into a DF
hashed_df = hashing.transform(newFrame)
hashed_df

DataFrame[_c0: string, Unnamed: 0: string, country: string, description: string, designation: string, points: string, price: string, province: string, region_1: string, region_2: string, taster_name: string, taster_twitter_handle: string, title: string, variety: string, winery: string, words: array<string>, filtered: array<string>, hashedValues: vector]

In [189]:
# Display new dataframe
# Format: (# of features, [array of numeric indices of each word, term frequency])
hashed_df.show(truncate=False)

+---+----------+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------------------------------------------------------------------------------------+------------------+-------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [190]:
hashed_df.select("hashedValues").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|hashedValues                                                                                                                                                                                                                                                                                                                                                                |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [196]:
hashed_df.select("description", "words","filtered", "hashValues").show()

AnalysisException: "cannot resolve '`hashValues`' given input columns: [winery, region_2, hashedValues, title, filtered, price, _c0, words, region_1, points, country, taster_name, variety, description, designation, taster_twitter_handle, province, Unnamed: 0];;\n'Project [description#2688, words#3003, filtered#3027, 'hashValues]\n+- AnalysisBarrier\n      +- Project [_c0#2685, Unnamed: 0#2686, country#2687, description#2688, designation#2689, points#2690, price#2691, province#2692, region_1#2693, region_2#2694, taster_name#2695, taster_twitter_handle#2696, title#2697, variety#2698, winery#2699, words#3003, filtered#3027, UDF(filtered#3027) AS hashedValues#3120]\n         +- Project [_c0#2685, Unnamed: 0#2686, country#2687, description#2688, designation#2689, points#2690, price#2691, province#2692, region_1#2693, region_2#2694, taster_name#2695, taster_twitter_handle#2696, title#2697, variety#2698, winery#2699, words#3003, UDF(words#3003) AS filtered#3027]\n            +- Project [_c0#2

In [193]:
# Fit the IDF on the data set
# This will calculate TF-IDF on each term
idf = IDF(inputCol="hashedValues", outputCol="features")
idfModel = idf.fit(hashed_df)
# rescaledData = idfModel.transform(hashed_df)

Py4JJavaError: An error occurred while calling o729.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 108.0 failed 1 times, most recent failure: Lost task 0.0 in stage 108.0 (TID 198, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$1: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at scala.collect

In [149]:
# Display the dataframe
rescaledData.select("sentences", "features").show(truncate=False)

NameError: name 'rescaledData' is not defined

# Feature Transformations

 Ins_NaiveBayes.ipynb

In [155]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

 

In [ ]:
 # Stop Spark
spark.stop()

## Reading with Pandas dataframe for clarity

In [59]:
# This is with Pandas dataframe
import pandas as pd
pddf = pd.read_csv('./InputData/winemag-data-130k-v2.csv')
pddf.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [61]:
pddf.drop(pddf.columns[[0]], axis=1, inplace=True) # drop that unnamed row numbers column
pddf.describe(include='all')

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
count,129908,129971,92506,129971.000000,120975.000000,129908,108724,50511,103727,98758,129971,129970,129971
unique,43,119955,37979,NaN,NaN,425,1229,17,19,15,118840,707,16757
top,US,This zesty red has pretty aromas that suggest ...,Reserve,NaN,NaN,California,Napa Valley,Central Coast,Roger Voss,@vossroger,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...,Pinot Noir,Wines & Winemakers
freq,54504,3,2009,NaN,NaN,36247,4480,11065,25514,25514,11,13272,222
mean,NaN,NaN,NaN,88.447138,35.363389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,3.039730,41.022218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,80.000000,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,86.000000,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,88.000000,25.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,91.000000,42.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 So, we care about description and variety. Hm, description... uniques and counts don't add up. if there are ~120k unique descriptions and 130k total, we have some duplicates. Let's take a look at one just to be sure they're actually just duplicates:

In [62]:
# looking at the duplicates, actually they are not duplicates. 
# Basically they start off with some of the same words in description, but they are not the same.

dups = pddf[pddf.duplicated('description')]
pddf_dups = dups.sort_values('description', ascending=False) 
pddf_dups

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
91976,Canada,"“Open and drink” the instructions should read,...",NaN,89,20.0,British Columbia,Okanagan Valley,NaN,Paul Gregutt,@paulgwine,Quails' Gate 2012 Chardonnay (Okanagan Valley),Chardonnay,Quails' Gate
129801,Italy,"“Il Bugiardo,” which is Italian for “the liar,...",Il Bugiardo,88,36.0,Veneto,Valpolicella Classico Superiore Ripasso,NaN,NaN,NaN,Buglioni 2009 Il Bugiardo (Valpolicella Class...,"Corvina, Rondinella, Molinara",Buglioni
34154,Italy,“Fresh and dry” is what the wine label adverti...,White Label,84,23.0,Piedmont,Gavi,NaN,NaN,NaN,La Scolca 2006 White Label (Gavi),Cortese,La Scolca
23271,Italy,‘Na Vota is a fresh and easygoing wine that is...,'Na Vota,85,18.0,Piedmont,Ruché di Castagnole Monferrato,NaN,NaN,NaN,Cantine Sant'Agata 2007 'Na Vota (Ruché di Ca...,Ruché,Cantine Sant'Agata
103573,New Zealand,this medium-bodied Sauvignon Blanc shows only ...,NaN,88,15.0,New Zealand,NaN,NaN,Joe Czerwinski,@JoeCz,Tussock Jumper 2016 Sauvignon Blanc (New Zealand),Sauvignon Blanc,Tussock Jumper
121126,US,lean and zesty Chardonnay with a burst of mout...,NaN,84,7.0,California,California,California Other,NaN,NaN,Eye Candy 2012 Chardonnay (California),Chardonnay,Eye Candy
109172,Austria,Zweigelt can do easy-drinking styles but in th...,Heideboden,90,26.0,Burgenland,NaN,NaN,Anne Krebiehl MW,@AnneInVino,Nittnaus Hans und Christine 2013 Heideboden Zw...,Zweigelt,Nittnaus Hans und Christine
95747,Italy,Zorzettig's precious Picolit dessert wine deli...,NaN,90,NaN,Northeastern Italy,Colli Orientali del Friuli,NaN,NaN,NaN,Zorzettig 2006 Picolit (Colli Orientali del Fr...,Picolit,Zorzettig
114892,US,Zinfandel fans will celebrate this powerfully ...,Old Vine Fanucchi-Wood Road Vineyard,92,55.0,California,Russian River Valley,Sonoma,NaN,NaN,Hartford Court 2011 Old Vine Fanucchi-Wood Roa...,Zinfandel,Hartford Court
35801,US,Zin fans should pick up this winery's entire r...,Clevenger,89,38.0,California,Paso Robles,Central Coast,Matt Kettmann,@mattkettmann,Peachy Canyon 2012 Clevenger Zinfandel (Paso R...,Zinfandel,Peachy Canyon


In [63]:
#Checked all the rows but they are no duplicates.Even though the description starts with same words, not all description is same.
pddf_dups.to_csv('output.csv', index=False, header=None)

In [65]:
# Get the count of non-null columns The count() method returns the number of non-NaN values in each column:
#Similarly, count(axis=1) returns the number of non-NaN values in each row.

# Count of description 129971 and Variety columns have same count
new_df = pddf.count()
new_df

country                  129908
description              129971
designation               92506
points                   129971
price                    120975
province                 129908
region_1                 108724
region_2                  50511
taster_name              103727
taster_twitter_handle     98758
title                    129971
variety                  129970
winery                   129971
dtype: int64

## Using Python Comparison Operators

In [73]:
# Same results only this time using python
df.filter(df["price"] < 200).show()

+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|_c0|  country|         description|         designation|points|price|         province|           region_1|         region_2|       taster_name|taster_twitter_handle|               title|           variety|             winery|
+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|  1| Portugal|This is ripe and ...|            Avidagos|    87| 15.0|            Douro|               null|             null|        Roger Voss|           @vossroger|Quinta dos Avidag...|    Portuguese Red|Quinta dos Avidagos|
|  2|       US|Tart and snappy, ...|                null|    87| 14.0|           Oregon|

In [74]:
df.filter( (df["price"] < 200) | (df['points'] > 80) ).show()

+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|_c0|  country|         description|         designation|points|price|         province|           region_1|         region_2|       taster_name|taster_twitter_handle|               title|           variety|             winery|
+---+---------+--------------------+--------------------+------+-----+-----------------+-------------------+-----------------+------------------+---------------------+--------------------+------------------+-------------------+
|  0|    Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|               Etna|             null|     Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|       White Blend|            Nicosia|
|  1| Portugal|This is ripe and ...|            Avidagos|    87| 15.0|            Douro|

In [76]:
df.filter(df["country"] == "Italy").show()

+---+-------+--------------------+--------------------+------+-----+-----------------+--------------------+--------+-------------+---------------------+--------------------+------------------+--------------------+
|_c0|country|         description|         designation|points|price|         province|            region_1|region_2|  taster_name|taster_twitter_handle|               title|           variety|              winery|
+---+-------+--------------------+--------------------+------+-----+-----------------+--------------------+--------+-------------+---------------------+--------------------+------------------+--------------------+
|  0|  Italy|Aromas include tr...|        Vulkà Bianco|    87| null|Sicily & Sardinia|                Etna|    null|Kerin O’Keefe|         @kerinokeefe|Nicosia 2013 Vulk...|       White Blend|             Nicosia|
|  6|  Italy|Here's a bright, ...|             Belsito|    87| 16.0|Sicily & Sardinia|            Vittoria|    null|Kerin O’Keefe|         @keri